# G-Drive

In [0]:
# # # MOUNTING DRIVE # # #
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Dependancies

In [0]:
import os
import pandas as pd
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

import datetime
import time

from scipy import stats


import xgboost as xgb
from xgboost.sklearn import XGBClassifier


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score,make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.utils import class_weight
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from nltk.classify.scikitlearn import SklearnClassifier

from imblearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTE

from mlxtend.classifier import StackingClassifier

from collections import Counter

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import tensorflow as tf
from keras.optimizers import sgd

import pickle

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


# Variables

In [0]:
fPath = '/content/gdrive/My Drive/AIML/Hackathon_04112020/wd/Files/'

# # # File Name # # # 
bond_ratings = 'bond_ratings.csv'
fund_allocations = 'fund_allocations.csv'
fund_config = 'fund_config.csv'
fund_ratios = 'fund_ratios.csv'
fund_specs = 'fund_specs.csv'
other_specs = 'other_specs.csv'
return_10year = 'return_10year.csv'
return_3year = 'return_3year.csv'
return_5year = 'return_5year.csv'

# # # Variables # # #
baseYear = 2020

# Loading files

In [0]:
dfBondRatings = pd.read_csv(fPath+bond_ratings)
dfFundAllocations = pd.read_csv(fPath+fund_allocations)
dfFundConfig = pd.read_csv(fPath+fund_config)
dfFundRatios = pd.read_csv(fPath+fund_ratios, thousands = ",")
dfFundSpecs = pd.read_csv(fPath+fund_specs)
dfOtherSpecs = pd.read_csv(fPath+other_specs, thousands = ",")
dfReturn10year = pd.read_csv(fPath+return_10year, thousands = ",")
dfReturn3year = pd.read_csv(fPath+return_3year, thousands = ",")
dfReturn5year = pd.read_csv(fPath+return_5year, thousands = ",")

dfSample = pd.read_csv(fPath+'sample_submission.csv') # Sample Submission

# # # Datframes Dictionary # # #
dictDataFrames = {'dfBondRatings':dfBondRatings,
                  'dfFundAllocations':dfFundAllocations,
                  'dfFundConfig':dfFundConfig,
                  'dfFundRatios':dfFundRatios,
                  'dfFundSpecs':dfFundSpecs,
                  'dfOtherSpecs':dfOtherSpecs,
                  'dfReturn10year':dfReturn10year,
                  'dfReturn3year':dfReturn3year,
                  'dfReturn5year':dfReturn5year}

dfOtherSpecs.rename(columns = {'2012_fund_return': '2012_return_fund', 
                                  '2013_category_return':'2013_return_category', 
                                  '2014_category_return': '2014_return_category',
                                  '2017_category_return' :'2017_return_category',
                                  'category_return_2015' : '2015_return_category',
                                  '1_month_fund_return' : 'fund_return_1months',
                                  '1_year_return_fund' : 'fund_return_1years',
                                  '3_months_return_category' : 'category_return_3month',
                                  'cash_percent_of_portfolio':'cash_percentage_of_portfolio',
                                  'stock_percent_of_portfolio' : 'stock_percentage_of_portfolio'}, inplace = True) 

# Functions


## DataFrame Functions


In [0]:
def funcValueCount(dfName,colName = ''):
  '''
    Displays the value counts of an specified dataframe and coloumn present in the dataframe.

  '''
  if dfName.__class__ == dict:
    for key,df in dfName.items():
      if key != 'dfReturn10year':
        print('\n')
        print('----- Dataframe {}: -----'.format(key))
        vc = df[colName].value_counts(dropna = False)
        print('Duplicate values in {} column are {}\n'.format(colName,vc[vc > 1]))
  else:
    vc = dfName[colName].value_counts(dropna = False)
    print('Duplicate values in {} column are {}\n'.format(colName,vc[vc > 1]))

def funcNullValueColumns(dfName):
  '''
  Prints the columns that have NaN values in them
  ''' 
  [col for col in pd.DataFrame(dfName).columns if pd.DataFrame(dfName)[col].isnull().any()]

def funcIsNull(dfName, colwise = True,threshold = 50, dropColumn = False):
    '''
    * Prints the NaN percentage in each column of the dataframe that is greater than the threshold.
    * Returns the list of column names whose percentage is greater than than the threshold.
    * if dropColumn is True drops the columns whose NaN percentage is high. 

    '''
    if dfName.__class__ == dict:
      for key,value in dfName.items():
        # print('\n----------Columns whose NaN percentage is greater than or equal to {} in {} dataframe\n'.format(threshold,key))
        series = value.isna().mean().round(4) * 100
        for identity,col in enumerate(series):
          if col > threshold:
            print('{}. {} --------- {}'.format(identity+1,series.index[identity],round(col,2)))
    else:
      if colwise == True:
        count = 1
        highNanCol = []
        dfNew = pd.DataFrame()
        series = dfName.isna().mean().round(4) * 100
        # print('Columns whose NaN percentage is greater than or equal to {}\n'.format(threshold))
        for identity,col in enumerate(series):
          if col >= threshold:
            # print('{}. {} --------- {}%'.format(count,series.index[identity],round(col,2)))
            count = count + 1
            highNanCol.append(series.index[identity])
        if dropColumn == True and len(highNanCol) >= 1:
          dfNew = dfName.drop(columns = highNanCol) 
          return dfNew
        else:
          return highNanCol
      else:
        highNanRow = []
        # print('Rows whose NaN percentage is greater than or equal to {}\n'.format(threshold))
        for i in range(len(dfName.index)) :
            temp = round((dfName.iloc[i].isnull().sum()/dfName.shape[1])*100,2)
            if temp >= threshold:
              highNanRow.append(dfName.iloc[i]['fund_id'])
        return highNanRow

def funcColName(dfName = dictDataFrames):
  '''
  Prints the column names in a dataframe.
  
  '''
  colName = []
  totCol = 1
  duplicateCol = []
  if dfName.__class__ == dict:
    for key,value in dfName.items():
      print('\n')
      print('----- The Columns in {} dataframe are: -----'.format(key))
      for count, col in enumerate(value.columns): 
        print('{}. {}'.format(count+1, col))
        totCol = totCol + 1
    print('************ Total Columns are {} ************'.format(totCol))
  else:
    for count,col in enumerate(dfName.columns.sort_values()):
      colName.append(col)
    return colName


def funcRenameColumn(dfName = dictDataFrames, newColName = 'uniqueID'):
  '''
  Renames the columns in all the dataframes and changes the Renamed column datatype to 'category'. Takes in a dictionary as an input.

  '''
  if dfName.__class__ == dict:
    for key,df in dfName.items():
      if 'tag' in df.columns:
        df.rename(columns={'tag':newColName}, inplace = True)
        df[newColName]=df[newColName].astype('category')
      elif 'id' in df.columns:
        df.rename(columns={'id':newColName}, inplace = True)
        df[newColName]=df[newColName].astype('category')

def funcDataFrameNames(dfName = dictDataFrames):
  '''
  Lists the names in the dataframe dictionary
  '''
  slNo = 1
  if dfName.__class__ == dict:
    for key,value in dfName.items():
      print('{}. {}'.format(slNo,key))
      slNo = slNo + 1

def funcConcatDF(dfName = dictDataFrames, uniqueID = 'uniqueID'):
  '''
  Concatenates the dataframesa and creates teh Master dataframe
  Removes duplicate columns.
  Renames the columns.
  '''
  newDF = pd.DataFrame()
  newDF_ = pd.DataFrame()
  listDuplicateCol = []
  listFundID = ['dfReturn10year','dfFundConfig','dfFundRatios']
  # DataFrames that have a 'fund_id' column
  if dfName.__class__ == dict:
    for key,df in dfName.items():
      if newDF.empty == True and key not in listFundID:
        newDF = df
      elif newDF.empty == False and key not in listFundID:
        newDF = pd.merge(newDF, df, on = uniqueID, how='inner')
      elif newDF_.empty == True and key in listFundID:
        newDF_ = df
      elif newDF_.empty == False and key in listFundID:
        newDF_ = pd.merge(newDF_, df, on = 'fund_id', how='inner')
  newDF = pd.merge(newDF, newDF_, on = 'uniqueID', how='inner')
  # Removes the duplicate columns 
  for colName in newDF.columns:
    if colName[-2:] == '_x':
      listDuplicateCol.append(colName)
  # Removes the uniqueID column by appending to the list listDuplicateCol
  listDuplicateCol.append('uniqueID')
  newDF = newDF.drop(columns = listDuplicateCol)
  # Renames the column by stripping _y at the end
  newDF.rename(columns = lambda x : str(x)[:-2] if x[-2:] == '_y' else x, inplace = True)
  del newDF_
  return newDF

def funcTestSetSplit(dfName):
  '''
  This function splits the actual test data from the combined dataframes
  '''
  dfActualTrain = dfName[dfName['greatstone_rating'].notna()]
  dfActualTest = dfName[dfName['greatstone_rating'].isnull()]
  return  dfActualTrain, dfActualTest

  #-----------------#-----------------#

def funcColStringSearch(dfName,word, lastWordSearch = 'any'):
  '''
  If lastWordSearch is 'last' it returns the column names that end with the 'word' hyperparameter else searches 'word' in the whole column
  The last argument only takes any, last, first
  '''
  listColName = ['fund_id']
  if lastWordSearch == 'any':
    for colName in dfName.columns:
      if word in colName:
        listColName.append(colName)
  elif lastWordSearch == 'last':
    searchwordLen = len(word) * -1
    for colName in dfName.columns:
      if colName[searchwordLen:] == word:
        listColName.append(colName)
  elif lastWordSearch == 'first':
    searchwordLen = len(word)
    for colName in dfName.columns:
      if colName[:searchwordLen] == word:
        listColName.append(colName)
  else:
    print("Wrong last argument in funcColStringSearch")
  return listColName

  #-----------------#-----------------#

def funcChangeColDataType(dfname, listColumn, type):
  '''
    Changes the datatype of the column
  '''
  for colName in dfName.columns:
    if dfName[colName] != np.float64:
      dfName[colName].astype('float64')

def funcColSearch(colName, dfName = dictDataFrames):
  '''
  Prints the dataframes that have the queried column
  '''
   for key,value in dfName.items():
     if colName in value.columns:
       print(key)


## ML Functions

### funcTopXOneHotEncode

In [0]:
def funcTopXOneHotEncode(dfName, feature, top = 20, dropOriginal = False, trainSet = True, sortAscending = False,listFeatureCategories = []):
  '''
  One hot encodes top or bottom X most frequent columns alone
  '''
  if trainSet == True:
    listTopXCategories = []
    if len(dfName[feature].unique()) >= top:
      listTopXCategories = dfName[feature].value_counts().sort_values(ascending = sortAscending).head(top).index
    else:
      listTopXCategories = dfName[feature].value_counts().sort_values(ascending = sortAscending).index
    for label in listTopXCategories:
      dfName[feature+'_'+label] = np.where(dfName[feature] == label,1,0)
    if dropOriginal == True:
      dfName = dfName.drop(columns = feature)
    return dfName, listTopXCategories
  else:
    for label in listFeatureCategories:
      dfName[feature+'_'+label] = np.where(dfName[feature] == label,1,0)
    if dropOriginal == True:
      dfName = dfName.drop(columns = feature)
    return dfName

### funcCustomCrossValidate()

In [0]:
def funcCustomCrossValidate(dfName, modelName, folds = 5):
  '''
  Custom K-folds implementation
  '''
  kf = KFold(n_splits = folds, shuffle = True)
  scores = []
  for i in range(folds):
      print('Split', folds)
      result = next(kf.split(dfName), None)
      x_train = dfName.iloc[result[0]]
      x_test = dfName.iloc[result[1]]
      y_train = y.iloc[result[0]]
      y_test = y.iloc[result[1]]
      model = modelName.fit(x_train,y_train)
      predictions = rf_reg.predict(x_test)
      scores.append(metrics.precision_score(np.array(y_test['greatstone_rating']), predictions, average = None).mean())
  return model, scores

### Model Functions


In [0]:
def funcCreateBalancedSampleWeights(y_train, largest_class_weight_coef):
  '''
  Create a list of weights associated with every row. It is used in certain ML models for penalizing imbalnce in the dataset.
  largest_class_weight_coef -> is the probability of the largest occuring class
  '''
    classes = y_train.unique()
    classes.sort()
    class_samples = np.bincount(y_train)
    total_samples = class_samples.sum()
    n_classes = len(class_samples)
    weights = total_samples / (n_classes * class_samples * 1.0)
    class_weight_dict = {key : value for (key, value) in zip(classes, weights)}
    class_weight_dict[classes[1]] = class_weight_dict[classes[1]] * largest_class_weight_coef
    sample_weights = [class_weight_dict[y] for y in y_train]
    return sample_weights

## Fast Functions

In [0]:
def funcListItemRemovals(listOriginal, listRemoveItems = []):
  '''
  Removes the items in the list
  '''
  for item in listRemoveItems:
    if item in listOriginal:
      listOriginal.remove(item)
    else:
      print(item, ' not available in list')
  return listOriginal

## Feature Engineering

### Feature Engineering

#### Feature Support Functions

In [0]:
def funcYearFeatureEngineering(dfNameSearch, df, trainFuncSet1,year = ''):
  '''
  This function help us to feature engineer the columns present in dfReturn10year, dfReturn5year, dfReturn3year.
  * Calculates the difference of Fund Mean catrgorywise and the fund value.
  * Calculates the difference between Fund Value and Category value.
  '''
  listSearch = ['alpha','beta','annual','r_squared', 'std', 'sharpe', 'treynor']
  listCategory = np.unique(np.array(trainFuncSet1['category'])).tolist()
  for name in listSearch:
    listTemp = funcColStringSearch(dfNameSearch,name)
    listTemp.remove('fund_id')
    for colNames in listTemp:
      if 'fund' in colNames:
        fund = colNames
      else:
        category = colNames
    tempName = year + name + '_YearReturn_FeatureEngineering'
    df[tempName] = df[fund] - df[category]
    tempName = year + name + '_FundYearReturn_FeatureEngineering'
    df[tempName] = df[fund] - trainFuncSet1[fund].mean(skipna = True)

    for colName in listTemp:
      if '_category' not in colName:
        tempName = colName + '_FundCategoryReturn_FeatureEngineering'
        df[tempName] = 0
        for category in listCategory:
          categoryMean = trainFuncSet1.loc[trainFuncSet1['category'] == category,[colName]].mean(skipna = True)
          df.loc[df['category']==category, tempName] =   df.loc[df['category']==category, colName] - categoryMean[0]
        df[tempName] = df[tempName].fillna(value= 0.00000001)

  return df

#### Engineering 1

In [0]:
def funcFeatureEngineering(dfName):
  '''
  Feature Engineering function 1
  '''

  # # # Treating Inception Date # # # 
  if 'inception_date' in dfName.columns and 'Days_Since_Inception' not in dfName.columns :
    dfName['inception_date'] = pd.to_datetime(dfName['inception_date'])
    dfName['Days_Since_Inception'] = (datetime.datetime.now()  - dfName['inception_date']).dt.days

  dfName['AverageAssets_FeatureEngineering'] = dfName['total_assets'] / (2020 - dfName['inception_date'].dt.year)
  dfName['AverageAssets_FeatureEngineering'] = dfName['AverageAssets_FeatureEngineering'].fillna(value= -999)
  dfName['yeild_expense_ratio'] = dfName['yield'] / dfName['fund_ratio_net_annual_expense']
  dfName['yeild_expense_ratio'] = dfName['yeild_expense_ratio'].fillna(value= -999)
  
  # # # Beta Feature Enginerring # # #
  listTempColName = funcColStringSearch(dfName, 'beta')
  listTempColName.remove('fund_id')
  for col in listTempColName:
    nameTemp = col + '_beta_FeatureEngineer'
    dfName[nameTemp]=-1
    dfName.loc[dfName[col]<1,nameTemp] = 0
    dfName.loc[dfName[col] == 1,nameTemp] = 1
    dfName.loc[dfName[col] > 1,nameTemp] = 2

  # # # Treating R Squared # # #
  listTempColName = funcColStringSearch(dfName, 'r_squared')
  listTempColName.remove('fund_id')
  for col in listTempColName:
    nameTemp = col + '_rSquare_FeatureEngineer'
    dfName[nameTemp]=-1
    if 'fund' in col:
      dfName.loc[((dfName[col]>=1) & (dfName[col] < 40)),nameTemp] = 0
      dfName.loc[((dfName[col]>=40) & (dfName[col] < .0)),nameTemp] = 1
      dfName.loc[((dfName[col]>=70) & (dfName[col] <= 100)),nameTemp] = 2
    else:
      dfName.loc[((dfName[col]>=.01) & (dfName[col] < .40)),nameTemp] = 0
      dfName.loc[((dfName[col]>=.40) & (dfName[col] < .70)),nameTemp] = 1
      dfName.loc[((dfName[col]>=.70) & (dfName[col] <= 1)),nameTemp] = 2

  listTemp = funcColName(dfFundAllocations)
  listTemp.remove('uniqueID')
  dfStdTemp = pd.DataFrame(dfName[listTemp].std(axis = 1, skipna = True), columns = ['PortfolioAllocationStd_FeatureEngineering'])
  dfName = pd.concat((dfName,dfStdTemp), axis = 1)

  dfName['LiquidityPercentage_FeatureEngineering'] = 100/(dfName['bond_percentage_of_porfolio'] + dfName['cash_percentage_of_portfolio'])

  listTemp = funcColStringSearch(dfOtherSpecs, '_fund')
  listTemp.remove('ytd_return_fund')
  listTemp.remove('fund_id')
  dfStdTemp = pd.DataFrame(dfName[listTemp].std(axis = 1, skipna = True), columns = ['FundYearsReturnStd_FeatureEngineering'])
  dfName = pd.concat((dfName,dfStdTemp), axis = 1)
  dfStdTemp = pd.DataFrame(dfName[listTemp].mean(axis = 1, skipna = True), columns = ['FundYearsReturnMean_FeatureEngineering'])
  dfName = pd.concat((dfName,dfStdTemp), axis = 1)

  listTemp = funcColStringSearch(dfOtherSpecs, '_category')
  listTemp.remove('ytd_return_category')
  listTemp.remove('fund_id')
  dfStdTemp = pd.DataFrame(dfName[listTemp].std(axis = 1, skipna = True), columns = ['CategoryYearsReturnStd_FeatureEngineering'])
  dfName = pd.concat((dfName,dfStdTemp), axis = 1)
  dfStdTemp = pd.DataFrame(dfName[listTemp].mean(axis = 1, skipna = True), columns = ['CategoryYearsReturnMean_FeatureEngineering'])
  dfName = pd.concat((dfName,dfStdTemp), axis = 1)

  return dfName


#### Engineering_2

In [0]:
def funcFeatureEngineering_2(dfName, trainFuncSet):
  '''
  Feature Engineering function 2
  '''
  df = dfName
  df['postiveYears_FeatureEngineer_2'] = -999
  df.loc[((df['years_up'] != -999) & (df['years_down'] != -999), 'postiveYears_FeatureEngineer_2' )] = df['years_up'] - df['years_down']

  df['tempMaxSelectionNA'] = 0.000001 #Used when all values in a col is zero for selecting the max
  colList = dfFundAllocations.columns.tolist()
  colList.remove('uniqueID')
  colList.append('tempMaxSelectionNA')
  df['maxAllocation_FeatureEngineer'] = df[colList].idxmax(axis=1)

  colList = ['stock_percentage_of_portfolio', 'bond_percentage_of_porfolio', 'cash_percentage_of_portfolio', 'tempMaxSelectionNA']
  df['maxPortfolioAllocation_FeatureEngineer'] = df[colList].idxmax(axis=1)

  colList = ['portfolio_convertable', 'portfolio_others','portfolio_preferred','tempMaxSelectionNA']
  df['maxPortfolioStockSelection_FeatureEngineer'] = df[colList].idxmax(axis=1)

  df['2010_FeatureEngineering'] = df['2010_return_fund'] - df['2010_return_category'] 
  df['2011_FeatureEngineering'] = df['2011_return_fund'] - df['2011_return_category'] 
  df['2012_FeatureEngineering'] = df['2012_return_fund'] - df['2012_return_category'] 
  df['2013_FeatureEngineering'] = df['2013_return_fund'] - df['2013_return_category'] 
  df['2014_FeatureEngineering'] = df['2014_return_fund'] - df['2014_return_category'] 
  df['2015_FeatureEngineering'] = df['2015_return_fund'] - df['2015_return_category'] 
  df['2016_FeatureEngineering'] = df['2016_return_fund'] - df['2016_return_category'] 
  df['2017_FeatureEngineering'] = df['2017_return_fund'] - df['2017_return_category'] 
  df['2018_FeatureEngineering'] = df['2018_return_fund'] - df['2018_return_category'] 

  df['2010_FundComp_FeatureEngineering'] = df['2010_return_fund'] - trainFuncSet['2010_return_fund'].mean(skipna = True)
  df['2011_FundComp_FeatureEngineering'] = df['2011_return_fund'] - trainFuncSet['2011_return_fund'].mean(skipna = True) 
  df['2012_FundComp_FeatureEngineering'] = df['2012_return_fund'] - trainFuncSet['2012_return_fund'].mean(skipna = True)
  df['2013_FundComp_FeatureEngineering'] = df['2013_return_fund'] - trainFuncSet['2013_return_fund'].mean(skipna = True)
  df['2014_FundComp_FeatureEngineering'] = df['2014_return_fund'] - trainFuncSet['2014_return_fund'].mean(skipna = True)
  df['2015_FundComp_FeatureEngineering'] = df['2015_return_fund'] - trainFuncSet['2015_return_fund'].mean(skipna = True)
  df['2016_FundComp_FeatureEngineering'] = df['2016_return_fund'] - trainFuncSet['2016_return_fund'].mean(skipna = True)
  df['2017_FundComp_FeatureEngineering'] = df['2017_return_fund'] - trainFuncSet['2017_return_fund'].mean(skipna = True)
  df['2018_FundComp_FeatureEngineering'] = df['2018_return_fund'] - trainFuncSet['2018_return_fund'].mean(skipna = True)

  df['Return1_FeatureEngineering'] = df['fund_return_1months'] - df['category_return_1month'] 
  df['Return2_FeatureEngineering'] = df['fund_return_3months'] - df['category_return_3month'] 
  df['Return3_FeatureEngineering'] = df['fund_return_1years'] - df['category_return_1year'] 

  df['Return1_FundComp_FeatureEngineering'] = df['fund_return_1months'] - trainFuncSet['fund_return_1months'].mean(skipna = True)
  df['Return2_FundComp_FeatureEngineering'] = df['fund_return_3months'] - trainFuncSet['fund_return_3months'].mean(skipna = True) 
  df['Return3_FundComp_FeatureEngineering'] = df['fund_return_1years'] - trainFuncSet['fund_return_1years'].mean(skipna = True) 

  df['Return4_FeatureEngineering'] = df['ytd_return_fund'] - df['ytd_return_category'] 
  df['AnnualExpenseComparison_FeatureEngineering'] = df['fund_ratio_net_annual_expense']  - df['category_ratio_net_annual_expense']

  df['Return4_FundComp_FeatureEngineering'] = df['ytd_return_fund'] - trainFuncSet['ytd_return_fund'].mean(skipna = True) 
  df['AnnualExpenseComparison_FundComp_FeatureEngineering'] = trainFuncSet['fund_ratio_net_annual_expense']  - df['fund_ratio_net_annual_expense'].mean(skipna = True) 

  listcol = ['2010_FeatureEngineering', 
              '2011_FeatureEngineering', 
              '2012_FeatureEngineering', 
              '2013_FeatureEngineering', 
              '2014_FeatureEngineering', 
              '2015_FeatureEngineering', 
              '2016_FeatureEngineering', 
              '2017_FeatureEngineering', 
              '2018_FeatureEngineering', 
              'Return1_FeatureEngineering', 
              'Return2_FeatureEngineering', 
              'Return3_FeatureEngineering', 
              'Return4_FeatureEngineering', 
              'AnnualExpenseComparison_FeatureEngineering']

  # df[listcol]= df[listcol].fillna(value = -999)

  df = funcYearFeatureEngineering(dfReturn10year, df,trainFuncSet1 = trainFuncSet, year = '10_')
  df = funcYearFeatureEngineering(dfReturn5year, df, trainFuncSet1 = trainFuncSet,year = '5_')
  df = funcYearFeatureEngineering(dfReturn3year, df, trainFuncSet1 = trainFuncSet,year = '3_')
  df['10_return_YearReturn_FeatureEngineering'] = df['10_years_return_fund'] - df ['10_years_return_category']
  df['5_return_YearReturn_FeatureEngineering'] = df['5_years_return_fund'] - df ['5_years_return_category']
  df['3_return_YearReturn_FeatureEngineering'] = df['fund_return_3years'] - df ['3_years_return_category']

  listTemp = funcColStringSearch(dfMainFeatures,'rating')
  listTemp.remove('fund_id')
  listTemp.remove('us_govt_bond_rating')
  listTemp.append('tempMaxSelectionNA')
  df['maxRating_FeatureEngineer'] = df[listTemp].idxmax(axis=1)

  listcol = ['2010_return_fund',
            '2011_return_fund',
            '2012_return_fund',
            '2013_return_fund',
            '2014_return_fund',
            '2015_return_fund',
            '2016_return_fund',
            '2017_return_fund',
            '2018_return_fund',
            'fund_return_1months', 'fund_return_3months', 'fund_return_1years', 'ytd_return_fund', 'fund_ratio_net_annual_expense'
            ]

  listCategory = np.unique(np.array(trainFuncSet['category'])).tolist()
  for colName in listcol:
    tempName = colName + '_FundCategoryReturn_FeatureEngineering'
    df[tempName] = 0
    for category in listCategory:
      categoryMean = trainFuncSet.loc[trainFuncSet['category'] == category,[colName]].mean(skipna = True)
      df.loc[df['category']==category, tempName] =   df.loc[df['category']==category, colName] - categoryMean[0]
    df[tempName] = df[tempName].fillna(value= 0.00000001)

  df = df.drop(columns = ['tempMaxSelectionNA'])

  return df

## Impute Function

### Section 1 

1.   funcImputeGroupBy()
2.   funcTreatingXyears()





#### funcTreatingXyears




In [0]:
def funcTreatingXyears(dfName,colSearchDF ,treatYears, trainSet = True, dictImpute = {}, dictImputeValue = '',trainNAValue = -1):
  '''
  Imputes 0 to all the funds whose age is less than treatYears years and Imputes mean for the remaining NaN values
  '''
  if trainSet == True:
    listTempColName = funcColStringSearch(colSearchDF, str(treatYears))
    # listColToRemove = funcColStringSearch(colSearchDF, 'category')
    # listTempColName = funcListItemRemovals(listTempColName,listColToRemove) 
    listTempColName.remove('fund_id') 
    dfName.loc[(baseYear - dfName['inception_date'].dt.year >= treatYears),listTempColName] = dfName.loc[(baseYear - dfName['inception_date'].dt.year >= treatYears),
                                                                                                                      listTempColName].fillna(value = -999)
    dfName.loc[(baseYear - dfName['inception_date'].dt.year < treatYears),listTempColName] = 0    
  else:
    listTempColName = funcColStringSearch(colSearchDF, str(treatYears))
    listTempColName.remove('fund_id') 
    dfName.loc[(baseYear - dfName['inception_date'].dt.year >= treatYears),listTempColName] = dfName.loc[(baseYear - dfName['inception_date'].dt.year >= treatYears),
                                                                                                                      listTempColName].fillna(value = -999) 
    dfName.loc[(baseYear - dfName['inception_date'].dt.year < treatYears),listTempColName] = 0   
  return dfName


### Section 2

#### funcImpute_()

In [0]:
'''
Treats Missing values of differnet dataframes
'''

# # # dfBondRatings # # #
def funcImpute_dfBondRatings(dfName,dictImpute = {},trainSet = True,trainNAValue = -1):
  if trainSet == True:
    listTempColName = funcColStringSearch(dfName,'_rating', lastWordSearch = 'last')
    listTempColName.remove('fund_id')
    dfName[listTempColName] = dfName[listTempColName].fillna(value= trainNAValue)
  else:
    listTempColName = funcColStringSearch(dfName,'_rating', lastWordSearch = 'last')
    listTempColName.remove('fund_id')
    dfName[listTempColName] = dfName[listTempColName].fillna(value= dictImpute['dfBondRatings'])
  return dfName

# # # dfFundAllocations # # #
def funcImpute_dfFundAllocations(dfName,dictImpute = {},trainSet = True,trainNAValue = -1):
  if trainSet == True:
    listTempColName = funcColName(dfFundAllocations)
    listTempColName.remove('uniqueID')
    dfName[listTempColName] = dfName[listTempColName].fillna(value= trainNAValue)
  else:
    listTempColName = funcColName(dfFundAllocations)
    listTempColName.remove('uniqueID')
    dfName[listTempColName] = dfName[listTempColName].fillna(value= dictImpute['dfFundAllocations'])
  return dfName

# # # dfFundRatios # # #
def funcImpute_dfFundRatios(dfName,dictImpute = {},trainSet = True,trainNAValue = -1):
  if trainSet == True:
    listTempColName = funcColName(dfFundRatios)
    listRemoveItems = ['fund_id','uniqueID', 'fund_ratio_net_annual_expense']
    listTempColName = funcListItemRemovals(listTempColName,listRemoveItems)
    dfName[listTempColName] = dfName[listTempColName].fillna(value= trainNAValue) 
  else:
    listTempColName = funcColName(dfFundRatios)
    listRemoveItems = ['fund_id','uniqueID', 'fund_ratio_net_annual_expense']
    listTempColName = funcListItemRemovals(listTempColName,listRemoveItems)
    dfName[listTempColName] = dfName[listTempColName].fillna(value= dictImpute['dfFundRatios'])
  return dfName

# # # dfFundSpecs # # # 
def funcImpute_dfFundSpecs(dfName,dictImpute = {},trainSet = True,trainNAValue = -1):
  if trainSet == True:
    # Categorical Columns Treating dfFundSpecs
    listTempColName = funcColName(dfFundSpecs)
    listRemoveItems = ['uniqueID', 'greatstone_rating']
    listTempColName = funcListItemRemovals(listTempColName,listRemoveItems)
    for colName in listTempColName:
      if colName in dfName.columns and dfName[colName].dtype == 'object':
        dfName.loc[dfName[colName].isnull(),colName] = 'UnKnown'    
    #Imputing Values
    listTempColName = ['return_ytd', 'yield', 'total_assets']
    dfName[listTempColName] = dfName[listTempColName].fillna(value= trainNAValue)
  else:
    # Categorical Columns Treating dfFundSpecs
    listTempColName = funcColName(dfFundSpecs)
    listRemoveItems = ['uniqueID', 'greatstone_rating']
    listTempColName = funcListItemRemovals(listTempColName,listRemoveItems)
    for colName in listTempColName:
      if colName in dfName.columns and dfName[colName].dtype == 'object':
        dfName.loc[dfName[colName].isnull(),colName] = 'UnKnown'   
    #Imputing Values   
    listTempColName = ['return_ytd', 'yield', 'total_assets']
    dfName[listTempColName] = dfName[listTempColName].fillna(value= dictImpute['dfFundSpecs'])
  return dfName

# # # dfOtherSpecs # # # 
def funcImpute_dfOtherSpecs(dfName,dictImpute = {},trainSet = True,trainNAValue = -1):
  listYear = [2010,2011,2012,2013,2014,2015,2016,2017,2018]
  if trainSet == True:
    for yearItem in listYear:
      listTempColName = funcColStringSearch(dfOtherSpecs,str(yearItem), 'first')
      listTempColName.remove('fund_id')
      dfName.loc[(dfName['inception_date'].dt.year > yearItem),listTempColName] = dfName.loc[(dfName['inception_date'].dt.year > yearItem),
                                                                                              listTempColName].fillna(value = 0)                                                                                                                                                              
    for yearItem in listYear:
      listTempColName = funcColStringSearch(dfOtherSpecs,str(yearItem), 'first')
      listTempColName.remove('fund_id')
      dfName[listTempColName] = dfName[listTempColName].fillna(value= trainNAValue)

    dfName.loc[dfName['inception_date'].dt.year == 2019, 'years_up'] = 0
    dfName.loc[dfName['inception_date'].dt.year == 2019, 'years_down'] = 0  

    listTempColName = ['category_return_3month', 'fund_return_1years', 'category_return_1year', 'fund_return_3years', 'ytd_return_fund', 'ytd_return_category',
                        'fund_return_3months', 'category_return_1month', 'fund_return_1months', 'cash_percentage_of_portfolio', 'bond_percentage_of_porfolio',
                        'portfolio_preferred', 'stock_percentage_of_portfolio','portfolio_others', 'portfolio_convertable',  'years_down', 'years_up']
    dfName[listTempColName] = dfName[listTempColName].fillna(value= trainNAValue)
  else:
    for yearItem in listYear:
      listTempColName = funcColStringSearch(dfOtherSpecs,str(yearItem), 'first')
      listTempColName.remove('fund_id')
      dfName.loc[(dfName['inception_date'].dt.year > yearItem),listTempColName] = dfName.loc[(dfName['inception_date'].dt.year > yearItem),
                                                                                                                                    listTempColName].fillna(value = 0)                                                                                                                       
    for yearItem in listYear:
      listTempColName = funcColStringSearch(dfOtherSpecs,str(yearItem), 'first')
      listTempColName.remove('fund_id')
      dfName[listTempColName] = dfName[listTempColName].fillna(value= trainNAValue)

    dfName.loc[dfName['inception_date'].dt.year == 2019, 'years_up'] = 0
    dfName.loc[dfName['inception_date'].dt.year == 2019, 'years_down'] = 0 

    listTempColName = ['category_return_3month', 'fund_return_1years', 'category_return_1year', 'fund_return_3years', 'ytd_return_fund', 'ytd_return_category',
                        'fund_return_3months', 'category_return_1month', 'fund_return_1months', 'cash_percentage_of_portfolio', 'bond_percentage_of_porfolio',
                        'portfolio_preferred', 'stock_percentage_of_portfolio','portfolio_others', 'portfolio_convertable',  'years_down', 'years_up']
    dfName[listTempColName] = dfName[listTempColName].fillna(value= dictImpute['dfOtherSpecs'])
  return dfName




#### funcImputeCentraltendancy()

In [0]:
def funcImputeCentraltendancy(df, trainSet = True, dictMain = {}):
  '''
  This function imputes missing values which were replaced by -999 using funcimpute_() functions with central tendancy of the column. 
  '''
  dfName = df
  if trainSet == True:
    dictCentralTendancy = {}
    for col in dfName.columns:
      if col != 'fund_id':
        naValue = dfName.loc[dfName[col]!=-999,col].mean()
        dfName.loc[dfName[col]==-999,[col]] = naValue
        dictCentralTendancy[col] = naValue
        #dictCentralTendancy.update({col: naValue})
    return dfName,dictCentralTendancy
  else:
    for col in dfName.columns:
      # Test Set
      if col != 'fund_id':
        naValue = dictMain[col]
        dfName.loc[dfName[col]==-999,[col]] = naValue
    return dfName

#### funcImputeMain()

In [0]:
def funcImputeMain(dfMainPreprocessed, boolTrainSetMain = True, dictImputeMain = {}):
  '''
  This funtion imputes values using the funcimpute_() function.
  '''
  dictMain = {}
  # # # Treating dfBondRatings # # #
  print('Treating dfBondRatings')
  naValue = -999
  dfMainPreprocessed = funcImpute_dfBondRatings(dfMainPreprocessed, dictImpute = dictImputeMain,trainSet = boolTrainSetMain,trainNAValue = naValue)
  dictMain.update({'dfBondRatings' : naValue})

  # # # Treating dfFundAllocations # # #
  print('Treating dfFundAllocations')
  naValue = -999                            # 9.09
  dfMainPreprocessed = funcImpute_dfFundAllocations(dfMainPreprocessed, dictImpute = dictImputeMain, trainSet = boolTrainSetMain,trainNAValue = naValue)
  dictMain.update({'dfFundAllocations' : naValue})

  # # # Treating dfFundRatios # # #
  print('Treating dfFundRatios')
  naValue = -999
  dfMainPreprocessed = funcImpute_dfFundRatios(dfMainPreprocessed, dictImpute = dictImputeMain, trainSet = boolTrainSetMain,trainNAValue = naValue)
  dictMain.update({'dfFundRatios' : naValue})

  # # # Treating dfFundSpecs # # #
  print('Treating dfFundRatios')
  naValue = -999
  dfMainPreprocessed = funcImpute_dfFundSpecs(dfMainPreprocessed, dictImpute = dictImputeMain, trainSet = boolTrainSetMain,trainNAValue = naValue)
  dictMain.update({'dfFundSpecs' : naValue})

  # # # Treating dfOtherSpecs # # #
  print('Treating dfOtherSpecs')
  naValue = -999
  dfMainPreprocessed = funcImpute_dfOtherSpecs(dfMainPreprocessed, dictImpute = dictImputeMain, trainSet = boolTrainSetMain,trainNAValue = naValue)
  dictMain.update({'dfOtherSpecs' : naValue})

  # # # Treating dfReturn10year # # #
  print('Treating dfReturn10year')
  dfMainPreprocessed = funcTreatingXyears(dfMainPreprocessed,dfReturn10year ,10, trainSet = boolTrainSetMain, 
                                          dictImpute = dictImputeMain, dictImputeValue = 'dfReturn10year',trainNAValue = -999)
  dictMain.update({'dfReturn10year' : naValue})

  # # # dfReturn3year # # #
  print('Treating dfReturn3year')
  dfMainPreprocessed = funcTreatingXyears(dfMainPreprocessed,dfReturn3year ,3, trainSet = boolTrainSetMain, 
                                          dictImpute = dictImputeMain, dictImputeValue = 'dfReturn3year',trainNAValue = -999)
  dictMain.update({'dfReturn3year' : naValue})

  # # # dfReturn5year # # #
  print('Treating dfReturn5year')
  dfMainPreprocessed = funcTreatingXyears(dfMainPreprocessed,dfReturn5year ,5, trainSet = boolTrainSetMain, 
                                          dictImpute = dictImputeMain, dictImputeValue = 'dfReturn5year',trainNAValue = -999)
  dictMain.update({'dfReturn5year' : naValue})
  if boolTrainSetMain == True:
    return dfMainPreprocessed, dictMain
  else:
    return dfMainPreprocessed

## Outlier Removal Function

In [0]:
def funcCapOutlierData(dfName, lowerQuantile =.25, upperQuantile = .75, trainSet = True, dictOutlier = {}):
  '''
  This function caps the outliers value with 25 percentile value and 75 percentile value
  '''
  if trainSet == True:
    dictOutlierTuple = {}
    if lowerQuantile > 1:
      lowerQuantile = lowerQuantile/100
    if upperQuantile > 1:
      upperQuantile = upperQuantile/100
    for col in dfName.columns:
        if (((dfName[col].dtype)=='float64') | ((dfName[col].dtype)=='int64')):
            percentiles = dfName[col].quantile([lowerQuantile,upperQuantile]).values
            lowerQuantileValue = percentiles[0]
            upperQuantileValue = percentiles[1]
            dfName.loc[dfName[col] <= lowerQuantileValue,col] = lowerQuantileValue
            dfName.loc[dfName[col] >= upperQuantileValue, col] = upperQuantileValue
            dictOutlierTuple[col] = (lowerQuantileValue, upperQuantileValue)
        # else:
        #     dfName[col]=dfName[col]
    return dfName, dictOutlierTuple
  else:
    for col in dfName.columns:
        if (((dfName[col].dtype)=='float64') | ((dfName[col].dtype)=='int64')):
          if col in dictOutlier.keys():
            lowerQuantileValue = dictOutlier[col][0]
            upperQuantileValue = dictOutlier[col][1]
            dfName.loc[dfName[col] <= lowerQuantileValue,col] = lowerQuantileValue
            dfName.loc[dfName[col] >= upperQuantileValue, col] = upperQuantileValue
          else:
            print(col,' not present in train dataset')
        # else:
        #     dfName[col]=dfName[col]
    return dfName

## funcCustomPipeLinePreprocessing






In [0]:
def funcCustomPipeLinePreprocessing(dfMain,trainFuncSetMain, trainSet = True, NaNThreshold = 50, removeRows = False,listColToRemoveCustom = [], dictImputeCustom = {}, dictOutlierCustom = {}, 
                                    dictOneHotCategoriesCustom={}, dictCentralTendancy = {}):
  '''
  * This function is used to ensure same preprocessing methods are applied in order for both the train data set and test dataset.
  * Returns the scalar values the were used for the Main dataframe to be used on the test dataset.
  '''
  dictOneHotCategories = {}
  if trainSet == True:
    # # # Columns To Remove # # #
    print('Removing columns')
    listColSameValue = dfMain.columns[dfMain.nunique() <= 1].tolist() # List of columns that have same value in them
    listColNaN = funcIsNull(dfMain,True,NaNThreshold) # List of columns that have NaN more than or equal to 50%
    listColToRemove = listColSameValue + listColNaN
    listColToRemove.append('fund_name')
    # listColToRemove.append('parent_company')
    # listColToRemove.append('category')
    dfMainPreprocessed = dfMain.drop(columns = listColToRemove)

    # # # Rows To Remove # # # 
    # ******** Not for the actual test dataset ********#
    listRowsToRemove = []
    if True == removeRows:
      listRowsToRemove = funcIsNull(dfMain,False,50)
      dfMainPreprocessed = dfMainPreprocessed[~dfMainPreprocessed['fund_id'].isin(listRowsToRemove)] # Removes the rows that have more than 50% NaN in them.

    print('One Hot Encoding Part 1')
    dfMainPreprocessed,listCategoriesTop = funcTopXOneHotEncode(dfMainPreprocessed, 'parent_company', top = 10, sortAscending = False,dropOriginal = False) # Top X OnehotEncoding 'parent_company' feature
    dfMainPreprocessed,listCategoriesBottom = funcTopXOneHotEncode(dfMainPreprocessed, 'parent_company', top = 10, sortAscending = True,dropOriginal = False) # Top X bottom
    dictOneHotCategories.update({'parent_company':(listCategoriesTop,listCategoriesBottom)})
    
    dfMainPreprocessed,listCategoriesTop = funcTopXOneHotEncode(dfMainPreprocessed, 'category', top = 10,sortAscending = False,dropOriginal = False) # Top X OnehotEncoding 'category' feature
    dfMainPreprocessed,listCategoriesBottom = funcTopXOneHotEncode(dfMainPreprocessed, 'category', top = 10, sortAscending = True,dropOriginal = False) # Top X bottom
    dictOneHotCategories.update({'category':(listCategoriesTop,listCategoriesBottom)})

    # # Treating Outliers # # #
    print('Treating Outlier')
    dfMainPreprocessed, dictOutlierCapTuple = funcCapOutlierData(dfMainPreprocessed,trainSet = True)

    # # # Feature Engineering # # #
    print('Feature Engineering 1')
    dfMainPreprocessed = funcFeatureEngineering(dfMainPreprocessed)
    
    # # # Imputation # # #
    print('Imputation')
    dfMainPreprocessed, dictMain = funcImputeMain(dfMainPreprocessed)

    # # # Feature Engineering # # # -> The below steps are after imputaion because we have imputed new categories
    print('Feature Engineering 2')
    dfMainPreprocessed = funcFeatureEngineering_2(dfMainPreprocessed,trainFuncSet = trainFuncSetMain)
    print('Done')

    dfMainPreprocessed = dfMainPreprocessed.drop(columns = ['parent_company', 'category'])

    print('One Hot Encoding Part 2')
    dfMainPreprocessed, listCategories= funcTopXOneHotEncode(dfMainPreprocessed, 'investment_class', dropOriginal = True) # Top X OnehotEncoding 'category' feature
    dictOneHotCategories.update({'investment_class':listCategories})

    labelencoder = LabelEncoder()
    dfMainPreprocessed['maxAllocation_FeatureEngineer'] = labelencoder.fit_transform(dfMainPreprocessed['maxAllocation_FeatureEngineer'])
    dictOneHotCategories.update({'maxAllocation_FeatureEngineer':labelencoder})
    
    labelencoder = LabelEncoder()
    dfMainPreprocessed['maxPortfolioAllocation_FeatureEngineer'] = labelencoder.fit_transform(dfMainPreprocessed['maxPortfolioAllocation_FeatureEngineer'])
    dictOneHotCategories.update({'maxPortfolioAllocation_FeatureEngineer':labelencoder})

    labelencoder = LabelEncoder()
    dfMainPreprocessed['maxRating_FeatureEngineer'] = labelencoder.fit_transform(dfMainPreprocessed['maxRating_FeatureEngineer'])
    dictOneHotCategories.update({'maxRating_FeatureEngineer':labelencoder})

    labelencoder = LabelEncoder()
    dfMainPreprocessed['maxPortfolioStockSelection_FeatureEngineer'] = labelencoder.fit_transform(dfMainPreprocessed['maxPortfolioStockSelection_FeatureEngineer'])
    dictOneHotCategories.update({'maxPortfolioStockSelection_FeatureEngineer':labelencoder})


    dfMainPreprocessed, listCategories = funcTopXOneHotEncode(dfMainPreprocessed, 'fund_size', dropOriginal = True) # Top X OnehotEncoding 'category' feature
    dictOneHotCategories.update({'fund_size':listCategories})
    # labelencoder = LabelEncoder()
    # dfMainPreprocessed['fund_size'] = labelencoder.fit_transform(dfMainPreprocessed['fund_size'])
    # dictOneHotCategories.update({'fund_size':labelencoder})

    listNaNCol = [col for col in pd.DataFrame(dfMainPreprocessed).columns if pd.DataFrame(dfMainPreprocessed)[col].isnull().any()]
    dfMainPreprocessed[listNaNCol] = dfMainPreprocessed[listNaNCol].fillna(value= -999)

    # # Central Tendancy Imputation # # #
    print('Imputing Central Tendancy')
    dfMainPreprocessed, dictMainCentralTendancy = funcImputeCentraltendancy(dfMainPreprocessed, trainSet = trainSet, dictMain = {})

    return dfMainPreprocessed, listColToRemove, dictMain, dictOutlierCapTuple, dictOneHotCategories, dictMainCentralTendancy

  else:

    # # # Removing Columns # # #
    dfMainPreprocessed = dfMain.drop(columns = listColToRemoveCustom)

    print('One Hot Encoding Part 1')
    dfMainPreprocessed = funcTopXOneHotEncode(dfMainPreprocessed, 'parent_company', dropOriginal = False,trainSet=False,listFeatureCategories=dictOneHotCategoriesCustom['parent_company'][0]) # Top X OnehotEncoding 'parent_company' feature
    dfMainPreprocessed = funcTopXOneHotEncode(dfMainPreprocessed, 'parent_company', dropOriginal = False,trainSet=False,listFeatureCategories=dictOneHotCategoriesCustom['parent_company'][1]) 
  
    dfMainPreprocessed = funcTopXOneHotEncode(dfMainPreprocessed, 'category',  dropOriginal = False,trainSet=False,listFeatureCategories=dictOneHotCategoriesCustom['category'][0]) # Top X OnehotEncoding 'category' feature
    dfMainPreprocessed = funcTopXOneHotEncode(dfMainPreprocessed, 'category',  dropOriginal = False,trainSet=False,listFeatureCategories=dictOneHotCategoriesCustom['category'][1])
    
    # # # Treating Outliers # # #
    print('Treating Outlier')
    dfMainPreprocessed = funcCapOutlierData(dfMainPreprocessed, trainSet = trainSet, dictOutlier = dictOutlierCustom)

    # # # Feature Engineering # # #
    print('Feature Engineering 1')
    dfMainPreprocessed = funcFeatureEngineering(dfMainPreprocessed)

    # # # Imputation # # #
    print('Imputation')
    dfMainPreprocessed = funcImputeMain(dfMainPreprocessed, boolTrainSetMain = trainSet, dictImputeMain = dictImputeCustom)

    print('Feature Engineering 2')
    dfMainPreprocessed = funcFeatureEngineering_2(dfMainPreprocessed,trainFuncSet = trainFuncSetMain)

    dfMainPreprocessed = dfMainPreprocessed.drop(columns = ['parent_company', 'category'])

    # The below steps are after imputaion because we have imputed new categories
    print('One Hot Encoding Part 2')
    dfMainPreprocessed = funcTopXOneHotEncode(dfMainPreprocessed, 'investment_class', dropOriginal = True,trainSet=False,listFeatureCategories=dictOneHotCategoriesCustom['investment_class']) # Top X OnehotEncoding 'category' feature

    dfMainPreprocessed = funcTopXOneHotEncode(dfMainPreprocessed, 'fund_size', dropOriginal = True,trainSet=False,listFeatureCategories=dictOneHotCategoriesCustom['fund_size']) # Top X OnehotEncoding 'category' feature

    labelEncoderTest = dictOneHotCategoriesCustom['maxAllocation_FeatureEngineer']
    dfMainPreprocessed['maxAllocation_FeatureEngineer'] = dfMainPreprocessed['maxAllocation_FeatureEngineer'].map(lambda s: '<unknown>' if s not in labelEncoderTest.classes_ else s)
    labelEncoderTest.classes_ = np.append(labelEncoderTest.classes_, '<unknown>')
    dfMainPreprocessed['maxAllocation_FeatureEngineer'] = labelEncoderTest.transform(dfMainPreprocessed['maxAllocation_FeatureEngineer'])
    
    labelEncoderTest = dictOneHotCategoriesCustom['maxPortfolioAllocation_FeatureEngineer']
    dfMainPreprocessed['maxPortfolioAllocation_FeatureEngineer'] = dfMainPreprocessed['maxPortfolioAllocation_FeatureEngineer'].map(lambda s: '<unknown>' if s not in labelEncoderTest.classes_ else s)
    labelEncoderTest.classes_ = np.append(labelEncoderTest.classes_, '<unknown>')
    dfMainPreprocessed['maxPortfolioAllocation_FeatureEngineer'] = labelEncoderTest.transform(dfMainPreprocessed['maxPortfolioAllocation_FeatureEngineer'])
    
    labelEncoderTest = dictOneHotCategoriesCustom['maxRating_FeatureEngineer']
    dfMainPreprocessed['maxRating_FeatureEngineer'] = dfMainPreprocessed['maxRating_FeatureEngineer'].map(lambda s: '<unknown>' if s not in labelEncoderTest.classes_ else s)
    labelEncoderTest.classes_ = np.append(labelEncoderTest.classes_, '<unknown>')
    dfMainPreprocessed['maxRating_FeatureEngineer'] = labelEncoderTest.transform(dfMainPreprocessed['maxRating_FeatureEngineer'])

    labelEncoderTest = dictOneHotCategoriesCustom['maxPortfolioStockSelection_FeatureEngineer']
    dfMainPreprocessed['maxPortfolioStockSelection_FeatureEngineer'] = dfMainPreprocessed['maxPortfolioStockSelection_FeatureEngineer'].map(lambda s: '<unknown>' if s not in labelEncoderTest.classes_ else s)
    labelEncoderTest.classes_ = np.append(labelEncoderTest.classes_, '<unknown>')
    dfMainPreprocessed['maxPortfolioStockSelection_FeatureEngineer'] = labelEncoderTest.transform(dfMainPreprocessed['maxPortfolioStockSelection_FeatureEngineer'])

    listNaNCol = [col for col in pd.DataFrame(dfMainPreprocessed).columns if pd.DataFrame(dfMainPreprocessed)[col].isnull().any()]
    dfMainPreprocessed[listNaNCol] = dfMainPreprocessed[listNaNCol].fillna(value= -999)
    
    # # Central Tendancy Imputation # # #
    print('Imputing Central Tendancy')
    dfMainPreprocessed = funcImputeCentraltendancy(dfMainPreprocessed, trainSet = False, dictMain = dictCentralTendancy)

    return dfMainPreprocessed

# Visualizations

In [0]:
# # # Class Imbalance # # #
dfMainLabels['greatstone_rating'].value_counts().plot(kind = 'bar', color = 'r', label = 'Count')
plt.title('Class Imbalance')
plt.xlabel('Classes')
plt.ylabel('Counts')
plt.legend()

In [0]:
def funcFeatureImportance(model, featureNums = -20):
  '''
  Visualizes important features
  '''
  features = cvTrain.columns
  importance = model.feature_importances_
  indices = np.argsort(importance)

  plt.figure(figsize=(10,15)) 
  plt.title('Feature Importances')
  plt.barh(range(len(indices[featureNums:])), importance[indices[featureNums:]], color='b', align='center')
  plt.yticks(range(len(indices[featureNums:])), [features[i] for i in indices[featureNums:]])
  plt.xlabel('Relative Importance')
  plt.show()

# Main

In [0]:
if __name__ == '__main__':

  if True == True:
    dfMain = pd.DataFrame()
    dfActualTest = pd.DataFrame()
    X_train = pd.DataFrame()
    X_test =  pd.DataFrame()

    scaler = StandardScaler()

    funcRenameColumn(dictDataFrames) # Renames tag and ID to uniqueID
    dfMain, dfActualTest= funcTestSetSplit(funcConcatDF(dictDataFrames)) # Function Inside an function
    dfMainFeatures = dfMain.loc[:, ~dfMain.columns.isin(['greatstone_rating'])] # Features are split from dfMain
    dfMainLabels = dfMain[['fund_id', 'greatstone_rating']] # Labels are split from dfMain

    X_train, X_test, y_train, y_test = train_test_split(dfMainFeatures, dfMainLabels, test_size=0.20, random_state=333) # Splitting dfMain into Test and Train
    X_trainOriginal = X_train
    print('-----------------------Train Set---------------------\n')
    X_train, listColToRemoveMain, dictImputeMain, dictOutlierCapTupleMain, dictOneHotCategoriesMain, dictMainCentralTendancy = funcCustomPipeLinePreprocessing(X_train, trainFuncSetMain = X_trainOriginal)
    X_train = X_train.drop(columns = ['fund_id', 'inception_date'])

    # # # Removing co-related columns # # #
    corr_matrix = X_train.corr().abs()
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    # Find index of feature columns with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
    to_drop
    toDropNew = []
    for item in to_drop:
      if  '_FeatureEngineer' not in item and '_fund' not in item:
        toDropNew.append(item)
    X_train = X_train.drop(columns = toDropNew)
    y_train = y_train.drop(columns = ['fund_id'])

    print('-----------------------Test Set---------------------\n')
    X_test = funcCustomPipeLinePreprocessing(X_test,trainSet = False,listColToRemoveCustom= listColToRemoveMain, dictImputeCustom=dictImputeMain, 
                                             dictOutlierCustom=dictOutlierCapTupleMain, dictOneHotCategoriesCustom = dictOneHotCategoriesMain, 
                                             dictCentralTendancy = dictMainCentralTendancy,
                                             trainFuncSetMain = X_trainOriginal)
    
    X_test = X_test.drop(columns = ['fund_id', 'inception_date'])
    X_test = X_test.drop(columns = toDropNew)
    y_test = y_test.drop(columns = ['fund_id'])

    print('-----------------------Actual Test Set---------------------\n')
    dfTest = dfSample.drop(columns = ['greatstone_rating'])
    productionDF = pd.merge(dfTest, dfActualTest, on = 'fund_id', how='inner')
    productionDF = productionDF.drop(columns = ['greatstone_rating'])
    productionDFFundID = productionDF['fund_id']
    productionDF = funcCustomPipeLinePreprocessing(productionDF,trainSet = False,listColToRemoveCustom= listColToRemoveMain, dictImputeCustom=dictImputeMain, 
                                             dictOutlierCustom=dictOutlierCapTupleMain,
                                             dictCentralTendancy = dictMainCentralTendancy, 
                                             dictOneHotCategoriesCustom = dictOneHotCategoriesMain,
                                             trainFuncSetMain = X_trainOriginal)
    
    productionDF = productionDF.drop(columns = ['fund_id', 'inception_date'])

  # # # Concatinating Train and Test Dataframes 
  cvTrain = pd.concat([pd.DataFrame(X_train),pd.DataFrame(X_test)],ignore_index=True)
  cvTest = pd.concat([y_train,y_test],ignore_index=True)

  # # # Standard Scalar
  print('-----------------------Standard Scalar---------------------\n')
  X_train = scaler.fit_transform(cvTrain)
  X_test = scaler.transform(X_test)
  cvTrain = scaler.fit_transform(cvTrain)
  productionDF = scaler.transform(productionDF)

  if True == True:
     print('-----------------------Stacking Model---------------------\n')
    strModelName = 'StackingModel'
    class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(y_train.values),
                                                  y_train['greatstone_rating']) 
    print('Cross Validation')
    sampleWeight = funcCreateBalancedSampleWeights(cvTest['greatstone_rating'], .34)
    classiRandomForest = RandomForestClassifier(n_estimators = 30, criterion = 'entropy',random_state = 143)
    classiKNN_1 = KNeighborsClassifier(n_neighbors = 301, weights = 'distance',algorithm = 'ball_tree')
    classiKNN_2 = KNeighborsClassifier(n_neighbors = 301,algorithm = 'ball_tree')
    classiNaiveBayes = GaussianNB()
    classiXGBoost_1 = XGBClassifier(
                    learning_rate =0.04,
                    n_estimators=500,
                    max_depth=17,
                    min_child_weight=1,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    objective= 'multi:softprob',
                    num_class = 6,
                    scale_pos_weight=1,
                    sample_weight = sampleWeight,
                    seed=27)
    classiXGBoost_2 = XGBClassifier(
                    learning_rate =0.04,
                    n_estimators=500,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    max_depth=17,
                    objective= 'multi:softprob',
                    num_class = 6,
                    sample_weight = sampleWeight,
                    seed=27)
    classiXGBoost_3 = XGBClassifier(
                    learning_rate =0.04,
                    n_estimators=700,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    max_depth=15,
                    objective= 'multi:softprob',
                    num_class = 6,
                    sample_weight = sampleWeight,
                    seed=27)
    classiXGBoost_4 = XGBClassifier(
                    learning_rate =0.04,
                    n_estimators=500,
                    max_depth=12,
                    colsample_bytree=0.8,
                    colsample_bylevel = 0.8,
                    colsample_bynode = 0.8,
                    objective= 'multi:softprob',
                    num_class = 6,
                    sample_weight = sampleWeight,
                    seed=27)
    classiXGBoost_5 = XGBClassifier(
                    learning_rate =0.04,
                    n_estimators=700,
                    colsample_bytree=0.8,
                    colsample_bylevel = 0.8,
                    colsample_bynode = 0.8,
                    max_depth=12,
                    objective= 'multi:softprob',
                    num_class = 6,
                    sample_weight = sampleWeight,
                    seed=27)
    classiAda = AdaBoostClassifier(
              DecisionTreeClassifier(max_depth=10, max_features = 'log2', min_samples_leaf = 71, 
                                     class_weight = {0:2.41109102,1:2.47371676,2:0.79294281,3:0.49118929,4:0.71703863,5: 2.04342273}, # Weights calculated using CreateBalancedSampleWeights
                                     random_state = 1234),n_estimators=500)
    classiLR = LogisticRegression(max_iter = 5000,dual = False, random_state=0)
    classiLR_1 = LogisticRegression(max_iter = 5000,solver = 'lbfgs',
                                    dual = False, random_state=0, 
                                    class_weight = {0:2.41109102,1:2.47371676,2:0.79294281,3:0.49118929,4:0.71703863,5: 2.04342273},
                                    C = .01,)
    classifier = StackingClassifier(classifiers=[classiAda,classiKNN_1, classiKNN_2,classiLR_1,classiXGBoost_1,classiXGBoost_2,classiXGBoost_4],
                                    use_probas=True,
                                    average_probas=False,
                                    verbose=2,
                                    use_features_in_secondary = False,
                                    meta_classifier=classiLR)

    print('---------Cross-Val-Score---------')
    scores = cross_val_score(classifier, pd.DataFrame(cvTrain), cvTest, cv = 5 , scoring = 'f1_macro', verbose=1)
    print('Mean {} , Std Deviation {}'.format(scores.mean(), scores.std()))

  if True == True:
    print('Fitting the model')
    classifier.fit(cvTrain, cvTest)
    print('Predecting the model')
    productionPrediction = classifier.predict(pd.DataFrame(productionDF))
    dictSubmission = {'fund_id':productionDFFundID, 'greatstone_rating':productionPrediction}
    dfSubmission = pd.DataFrame(dictSubmission)
    print('Saving File...')
    submissionPath = '/content/gdrive/My Drive/AIML/Hackathon_04112020/wd/Submissions/'+ datetime_object+'.csv'
    dfSubmission.to_csv(submissionPath, index = False)
    print('Submission File Saved in path')
  
  if True == False:
    print('Saving Model')
    modelName = '/content/gdrive/My Drive/AIML/Hackathon_04112020/wd/Model/'+strModelName+'_'+str(round(testPrecessionScore,2))+'_'+datetime_object+'.dat'
    pickle.dump(classifier, open(modelName, "wb"))
    print('Model Saved')

# Other